In [54]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
 
# save tokens to file, one dialog per line
def save_doc(lines, filename):
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()
 
# load text
fname = 'shakespeare_filtered.txt'
raw_text = load_doc(fname)
#print(raw_text)
 
# clean
tokens = raw_text.split()
#print(tokens)
raw_text = ' '.join(tokens)
 
# organize into sequences of characters
length = 40
sequences = list()
for i in range(length, len(raw_text)):
	# select sequence of tokens
	seq = raw_text[i-length:i+1]
	# store
	sequences.append(seq)
print(sequences[0])
print('Total Sequences: %d' % len(sequences))
 
# save sequences to file
out_filename = 'char_sequences.txt'
save_doc(sequences, out_filename)

From fairest creatures we desire increase
Total Sequences: 93633


In [56]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load
in_filename = 'char_sequences.txt'
raw_text = load_doc(in_filename)
lines = raw_text.split('\n')

# integer encode sequences of characters
chars = sorted(list(set(raw_text)))
mapping = dict((c, i) for i, c in enumerate(chars))
sequences = list()
for line in lines:
	# integer encode line
	encoded_seq = [mapping[char] for char in line]
	# store
	sequences.append(encoded_seq)

# vocabulary size
vocab_size = len(mapping)
print('Vocabulary Size: %d' % vocab_size)

# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
sequences = [to_categorical(x, num_classes=vocab_size) for x in X]
X = array(sequences)
y = to_categorical(y, num_classes=vocab_size)

Vocabulary Size: 61
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [43]:
from numpy import array
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Activation, Lambda
from keras.layers import LSTM

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load
in_filename = 'char_sequences.txt'
raw_text = load_doc(in_filename)
lines = raw_text.split('\n')

# integer encode sequences of characters
chars = sorted(list(set(raw_text)))
mapping = dict((c, i) for i, c in enumerate(chars))
sequences = list()
for line in lines:
	# integer encode line
	encoded_seq = [mapping[char] for char in line]
	# store
	sequences.append(encoded_seq)

# vocabulary size
vocab_size = len(mapping)
print('Vocabulary Size: %d' % vocab_size)

# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
sequences = [to_categorical(x, num_classes=vocab_size) for x in X]
X = array(sequences)
y = to_categorical(y, num_classes=vocab_size)

# define model
temp = 0.25
model = Sequential()
model.add(LSTM(100, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(vocab_size))
model.add(Lambda(lambda x: x / temp))
model.add(Activation('softmax'))
print(model.summary())
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, epochs=20, batch_size=10)

Vocabulary Size: 71
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_10 (LSTM)               (None, 100)               68800     
_________________________________________________________________
dense_10 (Dense)             (None, 71)                7171      
_________________________________________________________________
lambda_9 (Lambda)            (None, 71)                0         
_________________________________________________________________
activation_8 (Activation)    (None, 71)                0         
Total params: 75,971
Trainable params: 75,971
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/20
94141/94141 [==============================] - 1242s 13ms/step - loss: 2.1364 - acc: 0.3845
Epoch 2/20
94141/94141 [==============================] - 792s 8ms/step - loss: 1.8078 - acc: 0.4582
Epoch 3/20
94141/94141 [=================

In [50]:
from keras.models import load_model
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
 
# generate a sequence of characters with a language model
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
	in_text = seed_text
	# generate a fixed number of characters
	for _ in range(n_chars):
		# encode the characters as integers
		encoded = [mapping[char] for char in in_text]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# one hot encode
		encoded = to_categorical(encoded, num_classes=len(mapping))
		encoded = encoded.reshape(-1, encoded.shape[1], encoded.shape[2])
		# predict character
		yhat = model.predict_classes(encoded, verbose=0)
		# reverse map integer to character
		out_char = ''
		for char, index in mapping.items():
			if index == yhat:
				out_char = char
				break
		# append to input
		in_text += char
	return in_text

seed_line = "shall I compare thee to a summer's day?\n"

# test start of poem
print(generate_seq(model, mapping, 40, seed_line, 40*15))
print(generate_seq(model, mapping, 40, 'In me that thou shouldst that the sweet state, And therefore\n', 40*2))

shall I compare thee to a summer's day?
's alse the summer that thou shouldst bear, My sear have I will be it the world beare, The summer that the sweet summer that thee show, The likened buries the praise that the stain: So ill in the far the bright do not be song. 105 I like a wortated with the time and state, And therefore that the summer that thou art, In me that thou shouldst that the sweet state, And therefore that the summer that thou art, In me that thou shouldst that the sweet state, And therefore that the summer that thou art, In me that thou shouldst that the sweet state, And therefore that the summer that thou art, In me
In me that thou shouldst that the sweet state, And therefore
led the sweet summer that true, My dear strange that thou shouldst that the stre
